# Robust vehicle pre-allocation

This tutorial shows how to construct and solve the vehicle pre-allocation problem problem (Hao et al., 2020, https://onlinelibrary.wiley.com/doi/full/10.1111/poms.13143?casa_token=QjXxs7wyx8cAAAAA%3ADp6PFQDpob1UUd7qG8mSSlyEwmGjPrcjYaeSZxp8mmKaL1FUPfDVqcW4xt-HHLW_mjZlHIJOoETs2qo6). In this problem, there are $I$ supply nodes and $J$ demand nodes. For simplicity in this tutorial we assume that $I=1$. The operator needs to allocate vehicles before seeing the realization of the random demand. The problem can be formulated as follows:
$$
\begin{align}
\min_{\mathbf{x}, \mathbf{y}}\; & \left( \mathbf{c}-\mathbf{r} \right)^T\mathbf{x} + \mathbf{r}^T\mathbf{y} \nonumber \\

\text{s.t. } & \mathbf{y} \geq \mathbf{x}-\mathbf{d} \nonumber \\

&\mathbf{y} \geq \mathbf{0} \nonumber \\

&\sum_{j=1}^J x_j \leq q \nonumber \\

&\mathbf{x} \geq \mathbf{0} \nonumber \\
\end{align}
$$

where $\mathbf{x}\in\mathbb{R}^J$ is the vehicle allocation vector, i.e. we allocate $x_j$ vehicles from the supply node to the demand node $j$, $\mathbf{d}\in\mathbb{R}^J$ is the unknown demand vector, $\mathbf{c}\in\mathbb{R}^J$ is the cost vector. amd $\mathbf{r}\in\mathbb{R}^J$ is the revenue coefficient vector. $\mathbf{y}\in\mathbb{R}^J$ is a wait and see variable that depends on $\mathbf{d}$ and can be defined as follows:
$$
\mathbf{y} = \mathbf{y}^0 + \sum_{j=1}^J \mathbf{y}_j^k d_j
$$
where $\mathbf{y}^0, \left\{ \mathbf{y}_j^k\right\}_j\in\mathbb{R}^J$.

In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd

from lropt import Ellipsoidal, UncertainParameter, RobustProblem

We define the constants as shown below:

In [2]:
data = pd.read_csv('https://xiongpengnus.github.io/rsome/taxi_rain.csv')    # Load the data
demand = data.loc[:, 'Region1':'Region10']                                  # Demand data
data = demand.values                                                        # sample demand as an array
d_ub = demand.max().values                                                  # upper bound of demand
d_lb = demand.min().values 
I, J = 1, 10                                                                # Number of supply/demand nodes
r = np.array([4.50, 4.41, 3.61, 4.49, 4.38, 4.58, 4.53, 4.64, 4.58, 4.32])  # Revenue coefficients vector
c = 3 * np.ones(J)                                                          # Cost coefficients vector
q = 400                                                                     # Maximum vehicle supply

The problem is formulated and solved in the block below, using the Budget uncertainty set which is defined as follows:

Ellipsoidal uncertainty: $\{u \mid \| Au + b \|_p \leq \rho \}$

- $\rho$ : float, optional  
  * Ellipsoid scaling. Default 1.0. 
- $p$ : integer, optional  
  * Order of the norm. Default 2.
- $A$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b$ : np.array, optional
  * Relocation vector for u. Default None.

We directly add the inequality constraints on $\eta$ using the $\textit{lb},\textit{ub}$ optional arguments, which read as follows:
$$
lb \leq \mathbf{u} \leq ub \nonumber \\
$$

In [3]:
x = cp.Variable(J)                                                                  # Optimization variables, vehicles allocations
uncertainty_set = Ellipsoidal(p=np.inf, rho = np.max(d_ub), lb = d_lb, ub = d_ub)   # Ellipsoidal uncertainty set
d = UncertainParameter(J, uncertainty_set=uncertainty_set)                          # Uncertain parameters, unknown demand
y_0 = cp.Variable(J)                                                                # Helper optimization variables for y 
y_k = [cp.Variable(J) for _ in range(J)]                                            # Helper optimization variables for y
y = 0                                                                               # Optimization variables, bookeeping revenues
#Define y as a wait-and-see decisision variable
for j in range(J):
    y += y_k[j]
y += y_0


objective = cp.Minimize((c-r)@x + r@y)                                              # Problem objective
constraints = [
                y >= x-d,
                y >= 0,
                cp.sum(x) <= q,
                x >= 0,
              ]                                                                     # Problem constraints
prob = RobustProblem(objective=objective, constraints=constraints)                  # Define the problem using LROPT
prob.solve()                                                                        # Solve the problem using LROPT

-62.58973897871886